## 3. Monthly NVDI Composite Stats 

This notebook:

1. Loads the AOI
2. Loads the Sentinel-2 STAC manifest
3. Computes cloud-masked NDVI for each scene
4. Builds the monthly median NDVI composite
5. Saves:
   - GeoTIFF COG
   - Preview PNG
   - Monthly summary statistics

Running this notebook top-to-bottom **must produce the 3 final artifacts** required by the acceptance criteria.


### A. Library imports and path checks

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import rasterio

from thess_geo_analytics.utils.RepoPaths import RepoPaths
from thess_geo_analytics.geo.MonthlyCompositeBuilder import MonthlyCompositeBuilder, MonthlyCompositeConfig
from thess_geo_analytics.pipelines.ComputeMonthlyNdviStats import ComputeMonthlyNdviStats
from thess_geo_analytics.pipelines.BuildAssetsManifest import BuildAssetsManifest


### B. Parameters and Conifguration

In [2]:
MONTH = "2026-01"
AOI_ID = "el522"
AOI_PATH = RepoPaths.aoi("el522_thessaloniki.geojson")

print("Month:", MONTH)
print("AOI file:", AOI_PATH)


Month: 2026-01
AOI file: C:\Users\alexr\OneDrive\Desktop\thess-geo-analytics\aoi\el522_thessaloniki.geojson


### C. Inspect Manifest

In [ ]:
manifest_builder = BuildAssetsManifest()
manifest_builder.run()

In [3]:
manifest_path = RepoPaths.table(f"assets_manifest_{MONTH}.csv")
df = pd.read_csv(manifest_path)
df.head()


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\alexr\\OneDrive\\Desktop\\thess-geo-analytics\\outputs\\tables\\assets_manifest_2026-01.csv'

### D. Build monthly composite (very long)

In [ ]:
cfg = MonthlyCompositeConfig(max_scenes=None, verbose=False)

builder = MonthlyCompositeBuilder(
    aoi_path=AOI_PATH,
    aoi_id=AOI_ID,
    cfg=cfg,
)

out_tif, out_png = builder.run(MONTH)

print("Composite written:", out_tif)
print("Preview written:", out_png)


### E. NVDI Statistics computing

In [ ]:
stats_path = ComputeMonthlyNdviStats().run(MONTH, aoi_id=AOI_ID)
stats_path

### F. Plot Thessaloniki region median NVDI during january

In [ ]:
import matplotlib.pyplot as plt

with rasterio.open(out_tif) as ds:
    arr = ds.read(1).astype(float)
    nodata = ds.nodata
    if nodata is not None:
        arr = np.where(arr == nodata, np.nan, arr)

plt.figure(figsize=(10, 10))
plt.imshow(arr, vmin=-0.2, vmax=0.8, cmap="RdYlGn")
plt.colorbar(shrink=0.7)
plt.title(f"NDVI Composite {MONTH}")
plt.axis("off")
plt.show()
